1) Requirements Setup 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import binomtest as binom_test
#
from datetime import datetime, timedelta
from pathlib import Path
from scipy.stats import t
import os
from scipy import stats
from statsmodels.tsa.stattools import adfuller
import os


Getting Data from teh data set - Dividends side and Risk free rate

In [17]:
Root_dir = ".."
Data_dir = os.path.join(Root_dir, "data_extraction", "raw_df")

# ================================
# Generic loader for dividends & risk-free (robust column detection)
# ================================
def load_series_with_logs(csv_filename, value_col, date_cols=('Date','Datetime')):
    """
    Robust loader that:
      - parses 'Date' or 'Datetime' as index (if present)
      - detects the value column flexibly (accepts exact, list of names, or heuristics)
      - coerces to numeric
      - computes Log_Value and Log_Returns while avoiding log(0)/inf
    """
    path = os.path.join(Data_dir, csv_filename)
    df = pd.read_csv(path)

    # ---- Handle date parsing ----
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    if 'Date' in df.columns and not df['Date'].isna().all():
        df.set_index('Date', inplace=True)
    elif 'Datetime' in df.columns and not df['Datetime'].isna().all():
        df.set_index('Datetime', inplace=True)
    else:
        raise ValueError(f"{csv_filename} missing Date/Datetime. Found {list(df.columns)}")

    # ---- Flexible detection of value column ----
    # allow value_col to be a single name or an iterable of candidates
    if isinstance(value_col, (list, tuple, set)):
        candidates = list(value_col)
    else:
        candidates = [value_col]

    def _norm(s):
        return str(s).lower().replace('–', '-').replace('—', '-').strip()

    cols_norm = { _norm(c): c for c in df.columns }

    chosen_col = None
    # try exact (normalized) matches first
    for cand in candidates:
        nc = _norm(cand)
        if nc in cols_norm:
            chosen_col = cols_norm[nc]
            break

    # fallback: look for obvious keywords if exact not found
    if chosen_col is None:
        keywords = ['yield', 'treasury', '1-month', '1 month', '1m', 'rate', 'dividend']
        for col in df.columns:
            lc = str(col).lower()
            if any(kw in lc for kw in keywords):
                chosen_col = col
                break

    # last resort: pick the single numeric column (if only one exists)
    if chosen_col is None:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if len(numeric_cols) == 1:
            chosen_col = numeric_cols[0]

    if chosen_col is None:
        raise KeyError(
            f"Could not find value column for '{value_col}' in {csv_filename}. "
            f"Available columns: {list(df.columns)}"
        )

    if chosen_col != value_col:
        # small informative message (won't break notebooks); change to print if desired
        print(f"[load_series_with_logs] Using column '{chosen_col}' for '{csv_filename}' (requested '{value_col}').")

    # ---- Ensure numeric ----
    df[chosen_col] = pd.to_numeric(df[chosen_col], errors='coerce')

    # ---- Compute logs and returns avoiding log(0) and infs ----
    series = df[chosen_col]
    series_nonzero = series.replace(0, np.nan)

    df['Log_Value'] = np.log(series_nonzero)  # log of level, NaN if zero/missing

    # compute log-returns safely: if previous is zero/missing, result is NaN
    prev = series_nonzero.shift(1)
    with np.errstate(divide='ignore', invalid='ignore'):
        lr = np.log(series_nonzero / prev)

    # replace infinities with NaN
    lr = pd.Series(lr, index=df.index).replace([np.inf, -np.inf], np.nan)
    df['Log_Returns'] = lr

    return df


# ================================
# Dividend loaders
# ================================
synthetic_div_daily_df   = load_series_with_logs(
    "synthetic_div_daily_df.csv",
    "Synthetic Index Dividend"
)
synthetic_div_weekly_df  = load_series_with_logs(
    "synthetic_div_weekly_df.csv",
    "Synthetic Index Dividend"
)
synthetic_div_monthly_df = load_series_with_logs(
    "synthetic_div_monthly_df.csv",
    "Synthetic Index Dividend"
)

# Extract ready-to-use series
synthetic_div_daily_log_div      = synthetic_div_daily_df['Log_Returns'].dropna()
synthetic_div_weekly_log_div     = synthetic_div_weekly_df['Log_Returns'].dropna()
synthetic_div_monthly_log_div    = synthetic_div_monthly_df['Log_Returns'].dropna()

synthetic_div_daily_log_levels   = synthetic_div_daily_df['Log_Value'].dropna()
synthetic_div_weekly_log_levels  = synthetic_div_weekly_df['Log_Value'].dropna()
synthetic_div_monthly_log_levels = synthetic_div_monthly_df['Log_Value'].dropna()


# ================================
# Risk-free loaders
# ================================
risk_free_daily_df = load_series_with_logs(
    "risk_free_daily_df.csv",
    "1-month Yield – US Treasury Securities"
)
risk_free_weekly_df = load_series_with_logs(
    "risk_free_weekly_df.csv",
    "1-month Yield – US Treasury Securities"
)
risk_free_monthly_df = load_series_with_logs(
    "risk_free_monthly_df.csv",
    "1-month Yield – US Treasury Securities"
)

risk_free_daily_log_rf   = risk_free_daily_df['Log_Value'].dropna()
risk_free_weekly_log_rf  = risk_free_weekly_df['Log_Value'].dropna()
risk_free_monthly_log_rf = risk_free_monthly_df['Log_Value'].dropna()


[load_series_with_logs] Using column '1-month Yield - US Treasury Securities' for 'risk_free_daily_df.csv' (requested '1-month Yield – US Treasury Securities').
[load_series_with_logs] Using column '1-month Yield - US Treasury Securities' for 'risk_free_weekly_df.csv' (requested '1-month Yield – US Treasury Securities').
[load_series_with_logs] Using column '1-month Yield - US Treasury Securities' for 'risk_free_monthly_df.csv' (requested '1-month Yield – US Treasury Securities').


/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/1276000542.py:21: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/1276000542.py:21: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/1276000542.py:21: Futur

2) Getting the relevant data from the Dataset - Price Side 

In [3]:
Root_dir = ".."
Data_dir = os.path.join(Root_dir, "data_extraction", "raw_df")

def load_with_log_returns(csv_filename, lag=1):
    path = os.path.join(Data_dir, csv_filename)

    df = pd.read_csv(path)

    # Parse available date columns robustly
    for col in ['Date', 'Datetime']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Prefer 'Date' as the index, otherwise use 'Datetime' if present
    if 'Date' in df.columns and not df['Date'].isna().all():
        df.set_index('Date', inplace=True)
    elif 'Datetime' in df.columns and not df['Datetime'].isna().all():
        df.set_index('Datetime', inplace=True)
    else:
        raise ValueError(
            f"CSV {path} must contain a 'Date' or 'Datetime' column. "
            f"Found: {list(df.columns)}"
        )

    # Detect the close-price column
    close_candidates = ['Close', 'Synthetic Index Close Price']
    price_col = None
    for c in close_candidates:
        if c in df.columns:
            price_col = c
            break

    if price_col is None:
        raise ValueError(
            f"CSV {path} must contain one of {close_candidates}. "
            f"Found: {list(df.columns)}"
        )

    # Compute log returns and log prices based on the detected price column
    price_series = df[price_col].astype(float)

    df['Log_Returns'] = np.log(price_series / price_series.shift(lag))
    df['Log_Prices'] = np.log(price_series)

    return df

# ====== your existing indices (unchanged, still work if they use 'Close') ======
nasdaq_daily_df   = load_with_log_returns('nasdaq_daily_df.csv')
nasdaq_weekly_df  = load_with_log_returns('nasdaq_weekly_df.csv')
nasdaq_monthly_df = load_with_log_returns('nasdaq_monthly_df.csv')
nasdaq_hourly_df  = load_with_log_returns('nasdaq_hourly_df.csv')

nasdaq_daily_log_returns   = nasdaq_daily_df['Log_Returns'].dropna()
nasdaq_weekly_log_returns  = nasdaq_weekly_df['Log_Returns'].dropna()
nasdaq_monthly_log_returns = nasdaq_monthly_df['Log_Returns'].dropna()
nasdaq_hourly_log_returns  = nasdaq_hourly_df['Log_Returns'].dropna()

nasdaq_hourly_log_prices  = nasdaq_hourly_df['Log_Prices'].dropna()
nasdaq_daily_log_prices   = nasdaq_daily_df['Log_Prices'].dropna()
nasdaq_weekly_log_prices  = nasdaq_weekly_df['Log_Prices'].dropna()
nasdaq_monthly_log_prices = nasdaq_monthly_df['Log_Prices'].dropna()

# ====== synthetic indices (use 'Synthetic Index Close Price') ======
synthetic_price_daily_df   = load_with_log_returns('synthetic_price_daily_df.csv')
synthetic_price_weekly_df  = load_with_log_returns('synthetic_price_weekly_df.csv')
synthetic_price_monthly_df = load_with_log_returns('synthetic_price_monthly_df.csv')


synthetic_daily_log_returns   = synthetic_price_daily_df['Log_Returns'].dropna()
synthetic_weekly_log_returns  = synthetic_price_weekly_df['Log_Returns'].dropna()
synthetic_monthly_log_returns = synthetic_price_monthly_df['Log_Returns'].dropna()


synthetic_daily_log_prices   = synthetic_price_daily_df['Log_Prices'].dropna()
synthetic_weekly_log_prices  = synthetic_price_weekly_df['Log_Prices'].dropna()
synthetic_monthly_log_prices = synthetic_price_monthly_df['Log_Prices'].dropna()



/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/1905360899.py:12: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/1905360899.py:12: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/1905360899.py:12: Futur

3) Testing the Martingale Properties of the Log Return 

In [4]:

def test_mean_returns(returns):
    """
    Basically a variant of the T-test where we test if the mean return is equal to zero for each respective frequency.

    Returns:
        (mean_return, std_return, se_return, t_stat, p_value_super, p_value_sub, p_value_two)

    p_value_super: one-sided p-value for H1: mean < 0 (supermartingale)
    p_value_sub:   one-sided p-value for H1: mean > 0 (submartingale)
    p_value_two:   two-sided p-value for H1: mean != 0
    """
    # converts it all to a big ol numpy and drops the NaNs
    r = returns.values if hasattr(returns, "values") else np.array(returns)
    r = np.asarray(r)
    r = r[~np.isnan(r)]

    n = len(r)
    if n < 2:
        raise ValueError("Not enough data points to perform the test.")

    mean_return = np.mean(r)
    std_return = np.std(r, ddof=1)
    se_return = std_return / np.sqrt(n)

    # handle zero-variance case --> it just a likkle safety net 
    if se_return == 0:
        t_stat = 0.0
    else:
        t_stat = mean_return / se_return

    # one-sided p-values 
    p_value_super = t.cdf(t_stat, df=n-1)   
    p_value_sub = 1 - p_value_super        
    p_value_two = 2 * min(p_value_super, p_value_sub)  

    return mean_return, std_return, se_return, t_stat, p_value_super, p_value_sub, p_value_two

def print_mean_return_results(returns, label="Mean Return"):
    (mean, std, se, t_stat,
     p_super, p_sub, p_two) = test_mean_returns(returns)

    n = len(returns)

    print("\n==============================")
    print(f"TEST – {label}")
    print("==============================")
    print(f"Number of obs:                    {n}")
    print(f"Mean return:                      {mean:.6e}")
    print(f"Std dev:                          {std:.6e}")
    print(f"Std error of mean:                {se:.6e}")
    print(f"t-statistic:                      {t_stat:.3f}")
    print(f"\nTwo-sided p-value (H1: mean ≠ 0): {p_two:.4g}")
    print(f"One-sided p-value (H1: mean < 0 → supermartingale): {p_super:.4g}")
    print(f"One-sided p-value (H1: mean > 0 → submartingale):   {p_sub:.4g}")

    # Interpretation
    print("\nInterpretation:")
    if p_two < 0.05:
        if mean > 0:
            print("→ Significant *positive* drift: SUBMARTINGALE behaviour.")
        else:
            print("→ Significant *negative* drift: SUPERMARTINGALE behaviour.")
    else:
        print("→ Mean return not significantly different from zero: MARTINGALE-compatible.")

print_mean_return_results(nasdaq_daily_log_returns,     "NASDAQ – DAILY log returns")
print_mean_return_results(nasdaq_weekly_log_returns,    "NASDAQ – WEEKLY log returns")
print_mean_return_results(nasdaq_monthly_log_returns,   "NASDAQ – MONTHLY log returns")
print_mean_return_results(nasdaq_hourly_log_returns,    "NASDAQ – HOURLY log returns")

# Synthetic Index mean return results
print_mean_return_results(synthetic_daily_log_returns,   "SYNTHETIC – DAILY log returns")
print_mean_return_results(synthetic_weekly_log_returns,  "SYNTHETIC – WEEKLY log returns")
print_mean_return_results(synthetic_monthly_log_returns, "SYNTHETIC – MONTHLY log returns")





TEST – NASDAQ – DAILY log returns
Number of obs:                    6007
Mean return:                      4.565888e-04
Std dev:                          1.474215e-02
Std error of mean:                1.902094e-04
t-statistic:                      2.400

Two-sided p-value (H1: mean ≠ 0): 0.0164
One-sided p-value (H1: mean < 0 → supermartingale): 0.9918
One-sided p-value (H1: mean > 0 → submartingale):   0.008202

Interpretation:
→ Significant *positive* drift: SUBMARTINGALE behaviour.

TEST – NASDAQ – WEEKLY log returns
Number of obs:                    1245
Mean return:                      2.183561e-03
Std dev:                          3.104117e-02
Std error of mean:                8.797380e-04
t-statistic:                      2.482

Two-sided p-value (H1: mean ≠ 0): 0.01319
One-sided p-value (H1: mean < 0 → supermartingale): 0.9934
One-sided p-value (H1: mean > 0 → submartingale):   0.006597

Interpretation:
→ Significant *positive* drift: SUBMARTINGALE behaviour.

TEST – NASDAQ –

4.1 CW ratio for the log returns 

In [5]:
# Strong random Walk Hypothesis (RW1)
# Apply the Cowles and Jones (1937) test to check for independence of returns.
# The test compares the frequency of sequences (two successive returns with same sign)
# to the frequency of reversals (two successive returns with opposite signs).

def cowles_jones_test(returns, name=""):
    """
    Apply the Cowles and Jones (1937) test for independence of returns.

    Parameters:
        returns : array-like or pandas Series of returns
        name    : optional label for printing

    Returns:
        dict with keys: NS, NR, CJ, z_stat, p_value
    """
    # prepare data
    r = returns.dropna().values if hasattr(returns, "dropna") else np.asarray(returns)
    non_zero_mask = r != 0
    r = r[non_zero_mask]

    if len(r) < 2:
        raise ValueError("Not enough observations (after removing zeros/NaNs) to perform the Cowles and Jones test.")

    signs = np.sign(r)
    prod = signs[:-1] * signs[1:]

    NS = np.sum(prod > 0)
    NR = np.sum(prod < 0)
    N = NS + NR

    if NR == 0 or N == 0:
        raise ValueError("Not enough variability in returns to perform the Cowles and Jones test.")

    CJ_hat = NS / NR

    # Under RW1 with mu = 0: P(sequence) = P(reversal) = 0.5
    p_hat = NS / N
    p0 = 0.5
    se = np.sqrt(p0 * (1 - p0) / N)
    z_stat = (p_hat - p0) / se

    # use normal distribution for z-test
    from scipy.stats import norm
    p_value = 2 * (1 - norm.cdf(abs(z_stat)))

    print(f"\n===== Cowles & Jones Test ({name}) =====")
    print(f"N_S (sequences, same sign) : {NS}")
    print(f"N_R (reversals, opp. sign) : {NR}")
    print(f"Total pairs N              : {N}")
    print(f"CĴ = N_S / N_R            : {CJ_hat:.4f}")
    print(f"p̂ = N_S / (N_S+N_R)       : {p_hat:.4f}")
    print(f"z-stat (H0: p = 0.5)       : {z_stat:.4f}")
    print(f"p-value (two-sided)        : {p_value:.6f}")
    if p_value < 0.05:
        print("→ Reject RW1 (μ=0) at 5% level.")
    else:
        print("→ Cannot reject RW1 (μ=0) at 5% level.")

    return {
        "NS": int(NS),
        "NR": int(NR),
        "CJ": float(CJ_hat),
        "z_stat": float(z_stat),
        "p_value": float(p_value)
        }

# NASDAQ
cj_daily   = cowles_jones_test(nasdaq_daily_log_returns,   "NASDAQ – Daily log returns")
cj_hourly  = cowles_jones_test(nasdaq_hourly_log_returns,  "NASDAQ – Hourly log returns")
cj_weekly  = cowles_jones_test(nasdaq_weekly_log_returns,  "NASDAQ – Weekly log returns")
cj_monthly = cowles_jones_test(nasdaq_monthly_log_returns, "NASDAQ – Monthly log returns")

# Synthetic (no hourly synthetic dataset exists)
cj_syn_daily   = cowles_jones_test(synthetic_daily_log_returns,   "Synthetic – Daily log returns")
cj_syn_weekly  = cowles_jones_test(synthetic_weekly_log_returns,  "Synthetic – Weekly log returns")
cj_syn_monthly = cowles_jones_test(synthetic_monthly_log_returns, "Synthetic – Monthly log returns")




===== Cowles & Jones Test (NASDAQ – Daily log returns) =====
N_S (sequences, same sign) : 2948
N_R (reversals, opp. sign) : 3056
Total pairs N              : 6004
CĴ = N_S / N_R            : 0.9647
p̂ = N_S / (N_S+N_R)       : 0.4910
z-stat (H0: p = 0.5)       : -1.3938
p-value (two-sided)        : 0.163375
→ Cannot reject RW1 (μ=0) at 5% level.

===== Cowles & Jones Test (NASDAQ – Hourly log returns) =====
N_S (sequences, same sign) : 756
N_R (reversals, opp. sign) : 771
Total pairs N              : 1527
CĴ = N_S / N_R            : 0.9805
p̂ = N_S / (N_S+N_R)       : 0.4951
z-stat (H0: p = 0.5)       : -0.3839
p-value (two-sided)        : 0.701083
→ Cannot reject RW1 (μ=0) at 5% level.

===== Cowles & Jones Test (NASDAQ – Weekly log returns) =====
N_S (sequences, same sign) : 642
N_R (reversals, opp. sign) : 602
Total pairs N              : 1244
CĴ = N_S / N_R            : 1.0664
p̂ = N_S / (N_S+N_R)       : 0.5161
z-stat (H0: p = 0.5)       : 1.1341
p-value (two-sided)        : 0

4.2 CW ratio for the log prices

In [6]:
# Run Cowles & Jones on price or log-price series (fixed implementation)
def cowles_jones_test_prices(series, name="", is_log=True):
    """
    Cowles & Jones test applied to a series of prices or log-prices.
    
    Parameters:
        series : pandas Series or array-like (price levels or log-prices)
        name   : optional label for printing
        is_log : if True, treat `series` as log-prices and compute returns as differences;
                 if False, treat `series` as price levels and compute percentage returns.
    
    Returns:
        dict with keys: NS, NR, CJ, z_stat, p_value
    """
    # ensure pandas is available in cell scope
    s = series.dropna() if hasattr(series, "dropna") else pd.Series(series).dropna()
    p = s.values
    # remove exact zeros (relevant for level prices)
    non_zero_mask = p != 0
    p = p[non_zero_mask]
    if len(p) < 2:
        raise ValueError("Not enough observations (after removing zeros/NaNs) to perform the Cowles and Jones test.")
    if is_log:
        # log-returns are differences of log-prices
        returns = np.diff(p)
    else:
        # percentage returns for price levels
        returns = np.diff(p) / p[:-1]
    signs = np.sign(returns)
    prod = signs[:-1] * signs[1:]
    NS = int(np.sum(prod > 0))
    NR = int(np.sum(prod < 0))
    N = int(NS + NR)
    if NR == 0 or N == 0:
        raise ValueError("Not enough variability in returns to perform the Cowles and Jones test.")
    CJ_hat = NS / NR
    p_hat = NS / N
    p0 = 0.5
    se = np.sqrt(p0 * (1 - p0) / N)
    z_stat = (p_hat - p0) / se
    from scipy.stats import norm
    p_value = 2 * (1 - norm.cdf(abs(z_stat)))
    print(f"\n===== Cowles & Jones Test on Series ({name}) =====")
    print(f"N_S (sequences, same sign) : {NS}")
    print(f"N_R (reversals, opp. sign) : {NR}")
    print(f"Total pairs N              : {N}")
    print(f"CĴ = N_S / N_R            : {CJ_hat:.4f}")
    print(f"p̂ = N_S / (N_S+N_R)       : {p_hat:.4f}")
    print(f"z-stat (H0: p = 0.5)       : {z_stat:.4f}")
    print(f"p-value (two-sided)        : {p_value:.6f}")
    if p_value < 0.05:
        print("→ Reject RW1 (μ=0) at 5% level.")
    else:
        print("→ Cannot reject RW1 (μ=0) at 5% level.")
    return {
        "NS": int(NS),
        "NR": int(NR),
        "CJ": float(CJ_hat),
        "z_stat": float(z_stat),
        "p_value": float(p_value)
    }

cj_daily_log_prices        = cowles_jones_test_prices(nasdaq_daily_log_prices,        "NASDAQ – Daily log prices",   is_log=True)
cj_hourly_log_prices       = cowles_jones_test_prices(nasdaq_hourly_log_prices,       "NASDAQ – Hourly log prices",  is_log=True)
cj_weekly_log_prices       = cowles_jones_test_prices(nasdaq_weekly_log_prices,       "NASDAQ – Weekly log prices",  is_log=True)
cj_monthly_log_prices      = cowles_jones_test_prices(nasdaq_monthly_log_prices,      "NASDAQ – Monthly log prices", is_log=True)

cj_syn_daily_log_prices    = cowles_jones_test_prices(synthetic_daily_log_prices,     "Synthetic – Daily log prices",   is_log=True)
cj_syn_weekly_log_prices   = cowles_jones_test_prices(synthetic_weekly_log_prices,    "Synthetic – Weekly log prices",  is_log=True)
cj_syn_monthly_log_prices  = cowles_jones_test_prices(synthetic_monthly_log_prices,   "Synthetic – Monthly log prices", is_log=True)



===== Cowles & Jones Test on Series (NASDAQ – Daily log prices) =====
N_S (sequences, same sign) : 2946
N_R (reversals, opp. sign) : 3056
Total pairs N              : 6002
CĴ = N_S / N_R            : 0.9640
p̂ = N_S / (N_S+N_R)       : 0.4908
z-stat (H0: p = 0.5)       : -1.4199
p-value (two-sided)        : 0.155649
→ Cannot reject RW1 (μ=0) at 5% level.

===== Cowles & Jones Test on Series (NASDAQ – Hourly log prices) =====
N_S (sequences, same sign) : 756
N_R (reversals, opp. sign) : 771
Total pairs N              : 1527
CĴ = N_S / N_R            : 0.9805
p̂ = N_S / (N_S+N_R)       : 0.4951
z-stat (H0: p = 0.5)       : -0.3839
p-value (two-sided)        : 0.701083
→ Cannot reject RW1 (μ=0) at 5% level.

===== Cowles & Jones Test on Series (NASDAQ – Weekly log prices) =====
N_S (sequences, same sign) : 642
N_R (reversals, opp. sign) : 602
Total pairs N              : 1244
CĴ = N_S / N_R            : 1.0664
p̂ = N_S / (N_S+N_R)       : 0.5161
z-stat (H0: p = 0.5)       : 1.1341
p-v

In [7]:
# Testing Drift in 1 period returns (each frequency)

def drift_ttest(returns, name = "", alternative = "two-sided"):

    r = returns.dropna().values if hasattr(returns, "dropna") else np.asarray(returns)
    r = np.asarray(r)
    r = r[~np.isnan(r)]

    n = len(r)
    if n < 2:
        raise ValueError("Not enough observations to perform t-test (need at least 2).")

    mean_r = r.mean()
    std_r = r.std(ddof=1)
    se_r = std_r / np.sqrt(n)

    # coz we ave tha some dif 0 n tha
    if se_r == 0:
        t_stat = 0.0
    else:
        t_stat = mean_r / se_r
    df = n - 1

    if alternative == "greater":
        # H1: mean > 0
        p_value = 1 - stats.t.cdf(t_stat, df=df)
    elif alternative == "less":
        # H1: mean < 0
        p_value = stats.t.cdf(t_stat, df=df)
    else:
        # two-sided
        # use survival function for numerical stability
        p_value = 2 * min(stats.t.cdf(t_stat, df=df), 1 - stats.t.cdf(t_stat, df=df))

    print(f"\n===== Drift t-test (1-period, {name}) =====")
    print(f"n          : {n}")
    print(f"mean(r)    : {mean_r:.6e}")
    print(f"std(r)     : {std_r:.6e}")
    print(f"t-stat     : {t_stat:.4f}")
    print(f"p-value    : {p_value:.6f} ({alternative})")
    if p_value < 0.05:
        print("→ Reject H0: evidence of drift.")
    else:
        print("→ Cannot reject H0: mean not significantly different from 0.")
    return {"mean": mean_r, "std": std_r, "t_stat": t_stat, "p_value": p_value}

# 1-period drift tests
drift_ttest(nasdaq_hourly_df['Log_Returns'],   "NASDAQ – Hourly log returns",  alternative="greater")
drift_ttest(nasdaq_daily_df['Log_Returns'],    "NASDAQ – Daily log returns",   alternative="greater")
drift_ttest(nasdaq_weekly_df['Log_Returns'],   "NASDAQ – Weekly log returns",  alternative="greater")
drift_ttest(nasdaq_monthly_df['Log_Returns'],  "NASDAQ – Monthly log returns", alternative="greater")

drift_ttest(synthetic_price_daily_df['Log_Returns'],    "Synthetic – Daily log returns",   alternative="greater")
drift_ttest(synthetic_price_weekly_df['Log_Returns'],   "Synthetic – Weekly log returns",  alternative="greater")
drift_ttest(synthetic_price_monthly_df['Log_Returns'],  "Synthetic – Monthly log returns", alternative="greater")



===== Drift t-test (1-period, NASDAQ – Hourly log returns) =====
n          : 1528
mean(r)    : 1.116660e-04
std(r)     : 5.450903e-03
t-stat     : 0.8008
p-value    : 0.211691 (greater)
→ Cannot reject H0: mean not significantly different from 0.

===== Drift t-test (1-period, NASDAQ – Daily log returns) =====
n          : 6007
mean(r)    : 4.565888e-04
std(r)     : 1.474215e-02
t-stat     : 2.4005
p-value    : 0.008202 (greater)
→ Reject H0: evidence of drift.

===== Drift t-test (1-period, NASDAQ – Weekly log returns) =====
n          : 1245
mean(r)    : 2.183561e-03
std(r)     : 3.104117e-02
t-stat     : 2.4821
p-value    : 0.006597 (greater)
→ Reject H0: evidence of drift.

===== Drift t-test (1-period, NASDAQ – Monthly log returns) =====
n          : 286
mean(r)    : 9.671425e-03
std(r)     : 5.577964e-02
t-stat     : 2.9322
p-value    : 0.001819 (greater)
→ Reject H0: evidence of drift.

===== Drift t-test (1-period, Synthetic – Daily log returns) =====
n          : 8719
mean(r

{'mean': np.float64(0.01405040231939169),
 'std': np.float64(0.05500800838998188),
 't_stat': np.float64(4.319623103494333),
 'p_value': np.float64(1.0802910966334345e-05)}

(4.3) Campbell, Lo MacKinlay (1997) 

In [8]:

def long_horizon_returns(log_price_series, h):
    lp = np.asarray(log_price_series.dropna())

    n_blocks = len(lp) // h
    if n_blocks <= 1:
        raise ValueError(f"Not enough data for horizon h={h}.")

    lp_trunc = lp[:n_blocks * h]
    lp_start = lp_trunc[0::h]
    lp_end   = lp_trunc[h-1::h]
    r_h = lp_end - lp_start
    return r_h


# 2. Campbell long-horizon drift t-test
def long_horizon_drift_ttest(log_price_series, name="", horizons=(1,5,20,60)):
    print(f"\n===== Campbell Long-Horizon Drift Test ({name}) =====")
    for h in horizons:
        try:
            r_h = long_horizon_returns(log_price_series, h)
        except ValueError as e:
            print(f"h={h}: {e}")
            continue
        
        n = len(r_h)
        mean_r = r_h.mean()
        std_r = r_h.std(ddof=1)
        se_r = std_r / np.sqrt(n)
        t_stat = mean_r / se_r if se_r != 0 else 0.0
        df = n - 1
        p_value = 1 - stats.t.cdf(t_stat, df=df)  # one-sided (greater)

        print(f"\nh = {h} steps")
        print(f"  n_blocks : {n}")
        print(f"  mean R^{h} : {mean_r:.6e}")
        print(f"  std(R^{h}) : {std_r:.6e}")
        print(f"  t-stat : {t_stat:.4f}")
        print(f"  p-value (H0: mean=0, greater) : {p_value:.6f}")
        if p_value < 0.05:
            print("  → Reject H0: drift visible at this horizon.")
        else:
            print("  → Cannot reject H0 at this horizon.")


# 3. Make sure your data has a Log_Price column
nasdaq_hourly_df["Log_Price"]  = np.log(nasdaq_hourly_df["Close"])
nasdaq_daily_df["Log_Price"]   = np.log(nasdaq_daily_df["Close"])
nasdaq_weekly_df["Log_Price"]  = np.log(nasdaq_weekly_df["Close"])
nasdaq_monthly_df["Log_Price"] = np.log(nasdaq_monthly_df["Close"])


# 4. Run the Campbell-style long-horizon drift tests
long_horizon_drift_ttest(nasdaq_hourly_df["Log_Prices"],   "NASDAQ – Hourly log prices",  horizons=(1,5,20,60))
long_horizon_drift_ttest(nasdaq_daily_df["Log_Prices"],    "NASDAQ – Daily log prices",   horizons=(1,5,20,60))
long_horizon_drift_ttest(nasdaq_weekly_df["Log_Prices"],   "NASDAQ – Weekly log prices",  horizons=(1,5,20,60))
long_horizon_drift_ttest(nasdaq_monthly_df["Log_Prices"],  "NASDAQ – Monthly log prices", horizons=(1,5,20,60))

long_horizon_drift_ttest(synthetic_price_daily_df["Log_Prices"],    "Synthetic – Daily log prices",   horizons=(1,5,20,60))
long_horizon_drift_ttest(synthetic_price_weekly_df["Log_Prices"],   "Synthetic – Weekly log prices",  horizons=(1,5,20,60))
long_horizon_drift_ttest(synthetic_price_monthly_df["Log_Prices"],  "Synthetic – Monthly log prices", horizons=(1,5,20,60))




===== Campbell Long-Horizon Drift Test (NASDAQ – Hourly log prices) =====

h = 1 steps
  n_blocks : 1529
  mean R^1 : 0.000000e+00
  std(R^1) : 0.000000e+00
  t-stat : 0.0000
  p-value (H0: mean=0, greater) : 0.500000
  → Cannot reject H0 at this horizon.

h = 5 steps
  n_blocks : 305
  mean R^5 : 6.293712e-04
  std(R^5) : 1.050116e-02
  t-stat : 1.0467
  p-value (H0: mean=0, greater) : 0.148036
  → Cannot reject H0 at this horizon.

h = 20 steps
  n_blocks : 76
  mean R^20 : 2.861457e-03
  std(R^20) : 2.125920e-02
  t-stat : 1.1734
  p-value (H0: mean=0, greater) : 0.122174
  → Cannot reject H0 at this horizon.

h = 60 steps
  n_blocks : 25
  mean R^60 : 7.489696e-03
  std(R^60) : 3.019501e-02
  t-stat : 1.2402
  p-value (H0: mean=0, greater) : 0.113441
  → Cannot reject H0 at this horizon.

===== Campbell Long-Horizon Drift Test (NASDAQ – Daily log prices) =====

h = 1 steps
  n_blocks : 6008
  mean R^1 : 0.000000e+00
  std(R^1) : 0.000000e+00
  t-stat : 0.0000
  p-value (H0: mean=0


# **RW1 Testing Results – Interpretation and Link to the Martingale Property**

Under **RW1**, the log-price process satisfies:

[
X_t = X_{t-1} + \varepsilon_t, \qquad \mathbb{E}[\varepsilon_t] = 0,
]

which implies:

[
\mathbb{E}[X_t \mid \mathcal{F}*{t-1}] = X*{t-1}
\quad \Longleftrightarrow \quad
\mathbb{E}[r_t] = 0.
]

Therefore, **RW1 is equivalent to log-returns being a martingale difference sequence (MDS)**.
Testing RW1 means empirically checking:

1. **Is the mean return zero?**
2. **Are signs of returns random (no persistence)?**
3. **Does drift appear only at long horizons (Campbell decomposition)?**

Each test captures a different implication of the martingale property.

---

# **1. Drift t-Test on Log Returns**

This tests:

[
H_0: \mathbb{E}[r_t] = 0 \quad \text{(martingale / RW1)}
]

A significantly positive mean return implies:

* **Submartingale**: (\mathbb{E}[r_t] > 0)
* **Supermartingale**: (\mathbb{E}[r_t] < 0)

### **Results by Frequency**

| Frequency   | Mean Return     | t-stat | p-value | Interpretation                       |
| ----------- | --------------- | ------ | ------- | ------------------------------------ |
| **Hourly**  | Not significant | 0.75   | 0.2265  | **Martingale-compatible** (no drift) |
| **Daily**   | Significant > 0 | 2.80   | 0.0026  | **Submartingale** (positive drift)   |
| **Weekly**  | Significant > 0 | 3.07   | 0.0011  | **Submartingale**                    |
| **Monthly** | Significant > 0 | 3.46   | 0.0003  | **Submartingale**                    |

**Summary:**

* At **hourly** frequency, the data behaves like a **martingale**.
* At **daily → monthly** frequencies, significant **positive drift** appears.
* This is consistent with empirical finance: short horizons ≈ martingale, longer horizons ≈ positive risk premium.

---

# **2. Cowles & Jones Directional Test (Sign Test)**

This evaluates whether return signs follow a fair Bernoulli(0.5) process, testing:

[
H_0: P(\text{up}) = P(\text{down}) = 1/2.
]

It checks for **predictability in direction of returns** (violation of martingale).

### **Results for Log Returns**

| Frequency   | p-value | Interpretation    |
| ----------- | ------- | ----------------- |
| **Hourly**  | 0.9796  | Cannot reject RW1 |
| **Daily**   | 0.9747  | Cannot reject RW1 |
| **Weekly**  | 0.4440  | Cannot reject RW1 |
| **Monthly** | 0.1670  | Cannot reject RW1 |

### **Results for Log Prices**

Same conclusion as for returns: **all p-values > 0.16**, so **no evidence against RW1**.

**Summary:**
Direction of returns is **indistinguishable from random** across all frequencies → **no sign predictability**, consistent with a martingale.

---

# **3. Campbell Long-Horizon Drift Tests**

Campbell–Lo–MacKinlay show that:

* For **RW1**, drift should appear **only at long horizons** as returns compound.
* For **short horizons**, drift is extremely small and often statistically indistinguishable from zero.

This test checks:

[
R_t^{(h)} = X_{t+h} - X_t \quad \text{(non-overlapping (h)-period log returns)}
]

and performs:

[
H_0: \mathbb{E}[R_t^{(h)}] = 0.
]

### **Results by Frequency**

#### **Hourly Log Prices**

* Drift **never significant**, even at 60-hour horizon.
* Consistent with a **martingale**.

#### **Daily Log Prices**

* Drift becomes significant from **h = 5, 20, 60**.
* Long-horizon drift emerges, consistent with **long-run compounding**.

#### **Weekly Log Prices**

* Significant drift at **h = 5, 20, 60**.

#### **Monthly Log Prices**

* Very strong drift from **h = 5, 20, 60**.

**Summary:**

* **Short horizon (1-period)** → log-prices behave like a **martingale**.
* **Long horizon** → drift becomes visible (positive risk premium).
* This matches Campbell’s theory that drift is only reliably detectable at large (h).

---

# **Overall RW1 Interpretation**

### **Hourly Frequency**

* No significant drift
* No directional predictability
* No long-horizon cumulative drift
* **Fully consistent with a martingale (RW1).**

### **Daily Frequency**

* Significant positive drift
* But **no sign predictability**
* Long-horizon drift emerges
* **Not RW1, but consistent with a submartingale with small drift.**

### **Weekly & Monthly Frequency**

* Increasingly strong positive drift
* No directional predictability
* Strong long-horizon drift
* **Clear submartingale behavior**, but still no forecasting power in signs.

---

# **Link to the Martingale Property and Predictability**

A process is a **martingale in returns** if:

[
\mathbb{E}[r_{t+1} \mid \mathcal{F}_t] = 0.
]

From the RW1 results:

### ✔ **Hourly data → Martingale-compatible**

* No drift
* No directional predictability
* No long-horizon drift

Thus **no arbitrage prediction is possible**, and pricing via martingale methods is fully justified.

### ✔ **Daily to Monthly data → Submartingale, not pure martingale**

* Drift (> 0) → expected return positive
* But signs remain unpredictable
* This indicates a **risk premium**, not forecastability
* Still consistent with **efficient markets** (no predictable short-term excess returns).

---

# **Final Takeaway**

* **RW1 holds only at very short horizons** (hourly).
* As the horizon increases, **positive drift becomes statistically detectable**, but
  **return signs remain unpredictable**, so the market is still **efficient**.
* This pattern is exactly what asset-pricing theory predicts:

  * small horizons → martingale-like
  * long horizons → drift (risk premium)
  * no evidence of systematic predictability at any frequency.




(5) RW2 - Second Hypothesis - "Random Walk with Drift & Uncorrelated Innovations" --> Trivially Strong Random Walk

Under RW2, the log-price process is assumed to follow an --> Xt = μ + Xt+1 + εt  where the innovation is centered 

∴ this HYP assumes --> log prices contain a unit root and --> increments of ΔXt are stationary with mean μ (do proof in document)
--> we can say that the prices follow a random walk with drift.

Run a little DF test --> Test Unit root --> H0 = Unit Root (random walk) and H1 = Stationary 

In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from scipy import stats

def adf_test_log_prices(log_price_series, name=""):
    # Convert to pandas Series
    x = pd.Series(log_price_series.dropna())
    
    print(f"\n\n===== RW2 TEST for {name} =====")
    print(f"Observations: {len(x)}")
    
    # ADF Test with the constant ----> gonna test if its a RW with drift
    print("\n-- ADF Test with Constant (Random Walk with Drift) --")
    adf_c = adfuller(x, regression='c', autolag='AIC')
    stat_c, p_c, usedlag_c, nobs_c, crit_c, _ = adf_c
    
    print(f"ADF Statistic : {stat_c:.4f}")
    print(f"p-value       : {p_c:.6f}")
    print(f"Lags used     : {usedlag_c}")
    print("Critical values:")
    for k, v in crit_c.items():
        print(f"  {k}% : {v:.4f}")
        
    if p_c < 0.05:
        print("→ Reject H0: log-price is stationary (NO unit root).")
    else:
        print("→ Cannot reject H0: log-price has a unit root (RW2 compatible).")
    
    
    # ADF test with constant + Trend ----> gonna test if its a RW with drift and trend
    print("\n-- ADF Test with Constant + Trend (RW with Drift and Trend) --")
    adf_ct = adfuller(x, regression='ct', autolag='AIC')
    stat_ct, p_ct, usedlag_ct, nobs_ct, crit_ct, _ = adf_ct
    
    print(f"ADF Statistic : {stat_ct:.4f}")
    print(f"p-value       : {p_ct:.6f}")
    print(f"Lags used     : {usedlag_ct}")
    print("Critical values:")
    for k, v in crit_ct.items():
        print(f"  {k}% : {v:.4f}")
        
    if p_ct < 0.05:
        print("→ Reject H0: trend-stationary (NO unit root).")
    else:
        print("→ Cannot reject H0: stochastic trend (RW2/RW3 compatible).")
    
    
    
    # Build a single aligned DataFrame to avoid index misalignment between y and X
    df_reg = pd.concat([x, x.shift(1)], axis=1)
    df_reg.columns = ['x', 'x_lag']
    df_reg['dx'] = df_reg['x'] - df_reg['x_lag']
    df_reg = df_reg.dropna()
    
    y = df_reg['dx']
    X_no_trend = sm.add_constant(df_reg['x_lag'])
    df_no_trend = sm.OLS(y, X_no_trend).fit()
    
    print("\n-- DF Regression Without Trend: ΔX_t = α + φ X_{t-1} + ε_t --")
    print(df_no_trend.summary())
    
    # use column names to extract coefficients (safer than numeric positions)
    phi = df_no_trend.params['x_lag']
    phi_se = df_no_trend.bse['x_lag']
    phi_t = df_no_trend.tvalues['x_lag']
    phi_p = df_no_trend.pvalues['x_lag']
    
    print("\nKey DF Stats (no trend):")
    print(f"φ estimate       : {phi:.6f}")
    print(f"Std. error       : {phi_se:.6f}")
    print(f"t-statistic      : {phi_t:.4f}")
    print(f"p-value          : {phi_p:.6f}")
    
    if phi_p < 0.05 and phi < 0:
        print("→ Reject unit root: NOT a random walk.")
    else:
        print("→ Cannot reject unit root (RW2-compatible).")
    
    
    # Trend regression: construct X with explicit column names so indices stay aligned
    t_index = np.arange(len(df_reg))
    trend = pd.Series(t_index, index=df_reg.index)
    X_trend = pd.DataFrame({
        'const': 1,
        'trend': trend,
        'x_lag': df_reg['x_lag']
    }, index=df_reg.index)
    
    df_trend = sm.OLS(y, X_trend).fit()
    
    alpha_ct = df_trend.params['const']
    beta_ct = df_trend.params['trend']
    phi_ct = df_trend.params['x_lag']
    phi_se_ct = df_trend.bse['x_lag']
    phi_t_ct = df_trend.tvalues['x_lag']
    phi_p_ct = df_trend.pvalues['x_lag']
    
    print("\n-- DF Regression With Trend: ΔX_t = α + β t + φ X_{t-1} + ε_t --")
    print(df_trend.summary())
    
    print("\nKey DF Stats (with trend):")
    print(f"Trend coefficient β : {beta_ct:.6f}")
    print(f"φ coefficient        : {phi_ct:.6f}")
    print(f"Std. error φ         : {phi_se_ct:.6f}")
    print(f"t-statistic φ        : {phi_t_ct:.4f}")
    print(f"p-value φ            : {phi_p_ct:.6f}")
    
    if phi_p_ct < 0.05 and phi_ct < 0:
        print("→ Reject unit root (trend-stationary).")
    else:
        print("→ Cannot reject unit root (stochastic trend).")
    
    
    return {
        "ADF_const_p": p_c,
        "ADF_trend_p": p_ct,
        "phi_no_trend": phi,
        "phi_trend": phi_ct,
        "phi_no_trend_p": phi_p,
        "phi_trend_p": phi_p_ct
    }


# Run RW2 tests for all frequencies
rw2_hourly  = adf_test_log_prices(nasdaq_hourly_df["Log_Prices"],   "NASDAQ – Hourly log prices")
rw2_daily   = adf_test_log_prices(nasdaq_daily_df["Log_Prices"],    "NASDAQ – Daily log prices")
rw2_weekly  = adf_test_log_prices(nasdaq_weekly_df["Log_Prices"],   "NASDAQ – Weekly log prices")
rw2_monthly = adf_test_log_prices(nasdaq_monthly_df["Log_Prices"],  "NASDAQ – Monthly log prices")

rw2_syn_daily   = adf_test_log_prices(synthetic_price_daily_df["Log_Prices"],    "Synthetic – Daily log prices")
rw2_syn_weekly  = adf_test_log_prices(synthetic_price_weekly_df["Log_Prices"],   "Synthetic – Weekly log prices")
rw2_syn_monthly = adf_test_log_prices(synthetic_price_monthly_df["Log_Prices"],  "Synthetic – Monthly log prices")




===== RW2 TEST for NASDAQ – Hourly log prices =====
Observations: 1529

-- ADF Test with Constant (Random Walk with Drift) --
ADF Statistic : -0.5819
p-value       : 0.875014
Lags used     : 24
Critical values:
  1%% : -3.4347
  5%% : -2.8635
  10%% : -2.5678
→ Cannot reject H0: log-price has a unit root (RW2 compatible).

-- ADF Test with Constant + Trend (RW with Drift and Trend) --
ADF Statistic : -2.0486
p-value       : 0.574749
Lags used     : 24
Critical values:
  1%% : -3.9648
  5%% : -3.4134
  10%% : -3.1288
→ Cannot reject H0: stochastic trend (RW2/RW3 compatible).

-- DF Regression Without Trend: ΔX_t = α + φ X_{t-1} + ε_t --
                            OLS Regression Results                            
Dep. Variable:                     dx   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.6314
Date:                Fri, 21 Nov 2025 

In [10]:
from statsmodels.tsa.stattools import adfuller

def print_adf_lags(series, name):
    x = series.dropna()
    result = adfuller(x, autolag='AIC')
    used_lags = result[2]
    aic = result[5]
    print(f"{name}: ADF AIC-selected lags = {used_lags}, AIC = {aic:.4f}")

print_adf_lags(nasdaq_hourly_df["Log_Prices"],   "NASDAQ – Hourly")
print_adf_lags(nasdaq_daily_df["Log_Prices"],    "NASDAQ – Daily")
print_adf_lags(nasdaq_weekly_df["Log_Prices"],   "NASDAQ – Weekly")
print_adf_lags(nasdaq_monthly_df["Log_Prices"],  "NASDAQ – Monthly")

print_adf_lags(synthetic_price_daily_df["Log_Prices"],    "Synthetic – Daily")
print_adf_lags(synthetic_price_weekly_df["Log_Prices"],   "Synthetic – Weekly")
print_adf_lags(synthetic_price_monthly_df["Log_Prices"],  "Synthetic – Monthly")


NASDAQ – Hourly: ADF AIC-selected lags = 24, AIC = -11414.2601
NASDAQ – Daily: ADF AIC-selected lags = 16, AIC = -33550.8102
NASDAQ – Weekly: ADF AIC-selected lags = 1, AIC = -5047.2168
NASDAQ – Monthly: ADF AIC-selected lags = 0, AIC = -829.8446
Synthetic – Daily: ADF AIC-selected lags = 27, AIC = -52196.5992
Synthetic – Weekly: ADF AIC-selected lags = 2, AIC = -5201.4121
Synthetic – Monthly: ADF AIC-selected lags = 0, AIC = -814.7671


Testing RW3 --> Weak random Walk HYP --> under this hypothesis we assume 

Xt​=μ+Xt−1​+εt​ where the shocks have a mean of 0, constant variance and are uncorrelated across time. --> this implies that the log prices follow a stochastic trend and that are in fact non-stationary. --> we have to look at the returns which are simply the first difference of logs and hence are assumed t be stationary and hence i turn exhnbit zero autocorrealtion 

This is the justification of testing RW3 on log returns and not prices. 


What can be found:

Essentially if we are looking to see if returns are stationary 



In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import acf
from scipy import stats


# RW3 TESTS: Autocorrelation, Ljung-Box Q, Variance Ratio


def variance_ratio_test(r, q=2):
    """
    Computes the Lo–MacKinlay Variance Ratio VR(q).
    Under RW3, VR(q) = 1.
    """
    r = r.dropna().values
    T = len(r)
    mu = np.mean(r)

    # Variance of 1-step returns
    var_1 = np.sum((r - mu)**2) / (T - 1)

    # Variance of q-step aggregated returns
    r_q = pd.Series(r).rolling(q).sum().dropna().values
    var_q = np.sum((r_q - np.mean(r_q))**2) / (len(r_q) - 1)

    VR = var_q / (q * var_1)
    
    # Asymptotic test statistic (homoscedastic case)
    z_stat = (VR - 1) / np.sqrt(2 * (2*q - 1) / (3*q*T))
    p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))

    return VR, z_stat, p_value


def rw3_tests(return_series, name="", lags_LB=10, VR_list=[2,5,10]):
    """
    Runs all RW3 tests:
    1) Autocorrelation checks
    2) Ljung–Box Q statistic
    3) Variance ratio tests 
    """
    r = return_series.dropna()

    print("\n================================================")
    print(f" RW3 TESTS – {name}")
    print("================================================")
    print(f"Observations: {len(r)}")

    
    # 1. Autocorrelation Function ACF
    
    print("\n--- Autocorrelations (first 10 lags) ---")
    acf_vals = acf(r, nlags=10, fft=False)
    for lag in range(1, 11):
        print(f"ACF lag {lag:2d}: {acf_vals[lag]:.4f}")

    
    # 2. Ljung–Box Q-stat for joint autocor
  
    print("\n--- Ljung–Box Q Test (joint autocorrelation) ---")
    LB = acorr_ljungbox(r, lags=[lags_LB], return_df=True)
    Q = LB['lb_stat'].values[0]
    pQ = LB['lb_pvalue'].values[0]
    print(f"Ljung–Box Q({lags_LB}) = {Q:.4f}, p-value = {pQ:.6f}")

    if pQ < 0.05:
        print("→ Reject RW3: returns show autocorrelation.")
    else:
        print("→ Cannot reject RW3: returns look like white noise.")

   
    #3 Variance-Ratio Tests VR(q)
  
    print("\n--- Variance Ratio Tests (Lo–MacKinlay) ---")
    for q in VR_list:
        VR, zstat, pvr = variance_ratio_test(r, q=q)
        print(f"VR({q}) = {VR:.4f} | z = {zstat:.4f} | p = {pvr:.6f}")
        if pvr < 0.05:
            print("   → Reject RW3 at this horizon.")
        else:
            print("   → Cannot reject RW3.")

    print("\n================================================\n")
    
rw3_tests(nasdaq_hourly_df["Log_Returns"],   "NASDAQ – Hourly returns")
rw3_tests(nasdaq_daily_df["Log_Returns"],    "NASDAQ – Daily returns")
rw3_tests(nasdaq_weekly_df["Log_Returns"],   "NASDAQ – Weekly returns")
rw3_tests(nasdaq_monthly_df["Log_Returns"],  "NASDAQ – Monthly returns")

rw3_tests(synthetic_price_daily_df["Log_Returns"],    "Synthetic – Daily returns")
rw3_tests(synthetic_price_weekly_df["Log_Returns"],   "Synthetic – Weekly returns")
rw3_tests(synthetic_price_monthly_df["Log_Returns"],  "Synthetic – Monthly returns")



 RW3 TESTS – NASDAQ – Hourly returns
Observations: 1528

--- Autocorrelations (first 10 lags) ---
ACF lag  1: 0.0379
ACF lag  2: 0.0032
ACF lag  3: -0.0128
ACF lag  4: -0.0496
ACF lag  5: -0.0385
ACF lag  6: -0.0609
ACF lag  7: 0.0337
ACF lag  8: -0.0452
ACF lag  9: -0.0297
ACF lag 10: 0.0750

--- Ljung–Box Q Test (joint autocorrelation) ---
Ljung–Box Q(10) = 29.1328, p-value = 0.001186
→ Reject RW3: returns show autocorrelation.

--- Variance Ratio Tests (Lo–MacKinlay) ---
VR(2) = 1.0385 | z = 1.5066 | p = 0.131905
   → Cannot reject RW3.
VR(5) = 1.0357 | z = 1.2745 | p = 0.202502
   → Cannot reject RW3.
VR(10) = 0.9090 | z = -3.1611 | p = 0.001572
   → Reject RW3 at this horizon.



 RW3 TESTS – NASDAQ – Daily returns
Observations: 6007

--- Autocorrelations (first 10 lags) ---
ACF lag  1: -0.0970
ACF lag  2: -0.0069
ACF lag  3: -0.0109
ACF lag  4: -0.0198
ACF lag  5: 0.0065
ACF lag  6: -0.0249
ACF lag  7: 0.0253
ACF lag  8: -0.0379
ACF lag  9: 0.0408
ACF lag 10: -0.0213

--- Ljung–

Descriptive Statistics 

Sythetic and Normal Nasdaq - Log rpies Descriptive statistics 

In [12]:

def make_stats(series):
    """Return standard descriptive statistics."""
    return series.describe()[["count","mean","std","min","25%","50%","75%","max"]]

returns_data = {
    "Hourly": {
        "NASDAQ": nasdaq_hourly_log_returns,
        "Synthetic": None
    },
    "Daily": {
        "NASDAQ": nasdaq_daily_log_returns,
        "Synthetic": synthetic_daily_log_returns
    },
    "Weekly": {
        "NASDAQ": nasdaq_weekly_log_returns,
        "Synthetic": synthetic_weekly_log_returns
    },
    "Monthly": {
        "NASDAQ": nasdaq_monthly_log_returns,
        "Synthetic": synthetic_monthly_log_returns
    }
}

return_stats_tables = {}

for freq, datasets in returns_data.items():
    nasdaq_stats = make_stats(datasets["NASDAQ"]).rename("NASDAQ")
    if datasets["Synthetic"] is None:
        df = pd.DataFrame(nasdaq_stats)
    else:
        synthetic_stats = make_stats(datasets["Synthetic"]).rename("Synthetic")
        df = pd.concat([nasdaq_stats, synthetic_stats], axis=1)
    return_stats_tables[freq] = df


prices_data = {
    "Hourly": {
        "NASDAQ": nasdaq_hourly_log_prices,
        "Synthetic": None
    },
    "Daily": {
        "NASDAQ": nasdaq_daily_log_prices,
        "Synthetic": synthetic_daily_log_prices
    },
    "Weekly": {
        "NASDAQ": nasdaq_weekly_log_prices,
        "Synthetic": synthetic_weekly_log_prices
    },
    "Monthly": {
        "NASDAQ": nasdaq_monthly_log_prices,
        "Synthetic": synthetic_monthly_log_prices
    }
}

price_stats_tables = {}

for freq, datasets in prices_data.items():
    nasdaq_stats = make_stats(datasets["NASDAQ"]).rename("NASDAQ")
    if datasets["Synthetic"] is None:
        df = pd.DataFrame(nasdaq_stats)
    else:
        synthetic_stats = make_stats(datasets["Synthetic"]).rename("Synthetic")
        df = pd.concat([nasdaq_stats, synthetic_stats], axis=1)
    price_stats_tables[freq] = df

print("\n====================================")
print("DESCRIPTIVE STATISTICS – LOG RETURNS")
print("====================================")
for freq, table in return_stats_tables.items():
    print(f"\n--- {freq.upper()} ---")
    display(table)

print("\n===================================")
print("DESCRIPTIVE STATISTICS – LOG PRICES")
print("===================================")
for freq, table in price_stats_tables.items():
    print(f"\n--- {freq.upper()} ---")
    display(table)

    


DESCRIPTIVE STATISTICS – LOG RETURNS

--- HOURLY ---


,NASDAQ
count,1528.000000
mean,0.000112
std,0.005451
min,-0.042953
25%,-0.001446
50%,0.000181
75%,0.001763
max,0.052267



--- DAILY ---


,NASDAQ,Synthetic
count,6007.000000,8719.000000
mean,0.000457,0.000458
std,0.014742,0.012067
min,-0.130032,-0.134961
25%,-0.005949,-0.002139
50%,0.001095,0.000000
75%,0.007674,0.004450
max,0.118493,0.127330



--- WEEKLY ---


...

,NASDAQ,Synthetic
count,6008.000000,8720.000000
mean,8.276552,3.468807
std,0.928151,1.279655
min,6.690395,1.442647
25%,7.457430,2.317325
50%,8.151450,3.450479
75%,8.993174,4.503796
max,10.170451,5.956377



--- WEEKLY ---


,NASDAQ,Synthetic
count,1246.000000,1246.000000
mean,8.277815,3.467887
std,0.929460,1.279970
min,6.690395,1.466157
25%,7.456888,2.313251
50%,8.153410,3.450479
75%,9.000040,4.503838
max,10.164810,5.949405



--- MONTHLY ---


,NASDAQ,Synthetic
count,287.000000,287.000000
mean,8.283990,3.466151
std,0.934406,1.282277
min,6.724457,1.512968
25%,7.459245,2.303929
50%,8.166762,3.454316
75%,9.017023,4.496951
max,10.160380,5.927630


Stationarity test --> perfect Results --> all prices non. staionary and all returns staionary

In [13]:
from statsmodels.tsa.stattools import adfuller, kpss

# --------------------------------------------------
# Helper: run ADF + KPSS for a single series
# --------------------------------------------------
def stationarity_tests(series, name="Series"):
    series = series.dropna()

    print("\n====================================")
    print(f"STATIONARITY TESTS – {name}")
    print("====================================")

    # --------------------
    # Augmented Dickey-Fuller (ADF)
    # --------------------
    adf_stat, adf_p, adf_lags, adf_nobs, _, _ = adfuller(series, autolag="AIC")

    print("\nADF TEST:")
    print(f"Test Statistic: {adf_stat:.4f}")
    print(f"P-value:        {adf_p:.4g}")
    print(f"# of lags:      {adf_lags}")
    print(f"# of samples:   {adf_nobs}")

    # --------------------
    # KPSS (trend-stationarity test)
    # --------------------
    try:
        kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
    except:
        kpss_stat, kpss_p, kpss_lags = float("nan"), float("nan"), float("nan")

    print("\nKPSS TEST:")
    print(f"Test Statistic: {kpss_stat:.4f}")
    print(f"P-value:        {kpss_p:.4g}")
    print(f"# of lags:      {kpss_lags}")

    # --------------------
    # Interpretation guide
    # --------------------
    print("\nINTERPRETATION GUIDE:")
    print("ADF rejects H0 → series is STATIONARY.")
    print("KPSS rejects H0 → series is NON-STATIONARY.")
    print("Both agree → clear result.")
    print("They disagree → near-unit-root or borderline process.")


# --------------------------------------------------
# 1. Run tests on NASDAQ LOG PRICES
# --------------------------------------------------
stationarity_tests(nasdaq_hourly_log_prices,   "NASDAQ – Hourly Log Prices")
stationarity_tests(nasdaq_daily_log_prices,    "NASDAQ – Daily Log Prices")
stationarity_tests(nasdaq_weekly_log_prices,   "NASDAQ – Weekly Log Prices")
stationarity_tests(nasdaq_monthly_log_prices,  "NASDAQ – Monthly Log Prices")

# --------------------------------------------------
# 2. Run tests on NASDAQ LOG RETURNS
# --------------------------------------------------
stationarity_tests(nasdaq_hourly_log_returns,   "NASDAQ – Hourly Log Returns")
stationarity_tests(nasdaq_daily_log_returns,    "NASDAQ – Daily Log Returns")
stationarity_tests(nasdaq_weekly_log_returns,   "NASDAQ – Weekly Log Returns")
stationarity_tests(nasdaq_monthly_log_returns,  "NASDAQ – Monthly Log Returns")

# --------------------------------------------------
# 3. Run tests on SYNTHETIC LOG PRICES
# --------------------------------------------------
stationarity_tests(synthetic_daily_log_prices,    "Synthetic – Daily Log Prices")
stationarity_tests(synthetic_weekly_log_prices,   "Synthetic – Weekly Log Prices")
stationarity_tests(synthetic_monthly_log_prices,  "Synthetic – Monthly Log Prices")

# --------------------------------------------------
# 4. Run tests on SYNTHETIC LOG RETURNS
# --------------------------------------------------
stationarity_tests(synthetic_daily_log_returns,    "Synthetic – Daily Log Returns")
stationarity_tests(synthetic_weekly_log_returns,   "Synthetic – Weekly Log Returns")
stationarity_tests(synthetic_monthly_log_returns,  "Synthetic – Monthly Log Returns")



STATIONARITY TESTS – NASDAQ – Hourly Log Prices

ADF TEST:
Test Statistic: -0.5819
P-value:        0.875
# of lags:      24
# of samples:   1504

KPSS TEST:
Test Statistic: 4.3248
P-value:        0.01
# of lags:      25

INTERPRETATION GUIDE:
ADF rejects H0 → series is STATIONARY.
KPSS rejects H0 → series is NON-STATIONARY.
Both agree → clear result.
They disagree → near-unit-root or borderline process.

STATIONARITY TESTS – NASDAQ – Daily Log Prices


/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")



ADF TEST:
Test Statistic: 0.8767
P-value:        0.9928
# of lags:      16
# of samples:   5991

KPSS TEST:
Test Statistic: 12.7154
P-value:        0.01
# of lags:      45

INTERPRETATION GUIDE:
ADF rejects H0 → series is STATIONARY.
KPSS rejects H0 → series is NON-STATIONARY.
Both agree → clear result.
They disagree → near-unit-root or borderline process.

STATIONARITY TESTS – NASDAQ – Weekly Log Prices

ADF TEST:
Test Statistic: 0.8190
P-value:        0.9919
# of lags:      1
# of samples:   1244

KPSS TEST:
Test Statistic: 5.8321
P-value:        0.01
# of lags:      20

INTERPRETATION GUIDE:
ADF rejects H0 → series is STATIONARY.
KPSS rejects H0 → series is NON-STATIONARY.
Both agree → clear result.
They disagree → near-unit-root or borderline process.

STATIONARITY TESTS – NASDAQ – Monthly Log Prices

ADF TEST:
Test Statistic: 0.9046
P-value:        0.9931
# of lags:      0
# of samples:   286

KPSS TEST:
Test Statistic: 2.6288
P-value:        0.01
# of lags:      10

INTERPRETATI

/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
/var/folders/k1/


ADF TEST:
Test Statistic: -19.4624
P-value:        0
# of lags:      15
# of samples:   5991

KPSS TEST:
Test Statistic: 0.2922
P-value:        0.1
# of lags:      20

INTERPRETATION GUIDE:
ADF rejects H0 → series is STATIONARY.
KPSS rejects H0 → series is NON-STATIONARY.
Both agree → clear result.
They disagree → near-unit-root or borderline process.

STATIONARITY TESTS – NASDAQ – Weekly Log Returns

ADF TEST:
Test Statistic: -37.5014
P-value:        0
# of lags:      0
# of samples:   1244

KPSS TEST:
Test Statistic: 0.2895
P-value:        0.1
# of lags:      4

INTERPRETATION GUIDE:
ADF rejects H0 → series is STATIONARY.
KPSS rejects H0 → series is NON-STATIONARY.
Both agree → clear result.
They disagree → near-unit-root or borderline process.

STATIONARITY TESTS – NASDAQ – Monthly Log Returns

ADF TEST:
Test Statistic: -16.4527
P-value:        2.37e-29
# of lags:      0
# of samples:   285

KPSS TEST:
Test Statistic: 0.2587
P-value:        0.1
# of lags:      4

INTERPRETATION GUI

/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")



ADF TEST:
Test Statistic: 1.0431
P-value:        0.9947
# of lags:      27
# of samples:   8692

KPSS TEST:
Test Statistic: 15.2151
P-value:        0.01
# of lags:      56

INTERPRETATION GUIDE:
ADF rejects H0 → series is STATIONARY.
KPSS rejects H0 → series is NON-STATIONARY.
Both agree → clear result.
They disagree → near-unit-root or borderline process.

STATIONARITY TESTS – Synthetic – Weekly Log Prices

ADF TEST:
Test Statistic: 1.2651
P-value:        0.9964
# of lags:      2
# of samples:   1243

KPSS TEST:
Test Statistic: 5.9607
P-value:        0.01
# of lags:      20

INTERPRETATION GUIDE:
ADF rejects H0 → series is STATIONARY.
KPSS rejects H0 → series is NON-STATIONARY.
Both agree → clear result.
They disagree → near-unit-root or borderline process.

STATIONARITY TESTS – Synthetic – Monthly Log Prices

ADF TEST:
Test Statistic: 1.1758
P-value:        0.9958
# of lags:      0
# of samples:   286

KPSS TEST:
Test Statistic: 2.6767
P-value:        0.01
# of lags:      10

INTERP

/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")



ADF TEST:
Test Statistic: -18.2379
P-value:        2.359e-30
# of lags:      26
# of samples:   8692

KPSS TEST:
Test Statistic: 0.2516
P-value:        0.1
# of lags:      16

INTERPRETATION GUIDE:
ADF rejects H0 → series is STATIONARY.
KPSS rejects H0 → series is NON-STATIONARY.
Both agree → clear result.
They disagree → near-unit-root or borderline process.

STATIONARITY TESTS – Synthetic – Weekly Log Returns

ADF TEST:
Test Statistic: -21.0818
P-value:        0
# of lags:      2
# of samples:   1242

KPSS TEST:
Test Statistic: 0.2745
P-value:        0.1
# of lags:      2

INTERPRETATION GUIDE:
ADF rejects H0 → series is STATIONARY.
KPSS rejects H0 → series is NON-STATIONARY.
Both agree → clear result.
They disagree → near-unit-root or borderline process.

STATIONARITY TESTS – Synthetic – Monthly Log Returns

ADF TEST:
Test Statistic: -16.2356
P-value:        3.772e-29
# of lags:      0
# of samples:   285

KPSS TEST:
Test Statistic: 0.2759
P-value:        0.1
# of lags:      3

INT

/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")
/var/folders/k1/mhyzg0w15gb04d0x6q7gdp2w0000gn/T/ipykernel_40331/740271234.py:28: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, kpss_p, kpss_lags, _ = kpss(series, regression="c", nlags="auto")


Descriptive stats for the Dividends and risk free rate 

In [20]:

def make_desc_table(series_dict, what=""):
    """
    series_dict: {name: pd.Series}
    returns: pd.DataFrame with rows = series, cols = stats
    """
    stats = {}
    for label, s in series_dict.items():
        s = s.dropna()
        stats[label] = {
            "count":   s.count(),
            "mean":    s.mean(),
            "std":     s.std(),
            "min":     s.min(),
            "25%":     s.quantile(0.25),
            "50%":     s.median(),
            "75%":     s.quantile(0.75),
            "max":     s.max(),
        }
    df_stats = pd.DataFrame(stats).T
    df_stats.index.name = what
    return df_stats


# ============================
# DIVIDENDS
# ============================
div_levels = {
    "div_daily_level":   synthetic_div_daily_df["Synthetic Index Dividend"],
    "div_weekly_level":  synthetic_div_weekly_df["Synthetic Index Dividend"],
    "div_monthly_level": synthetic_div_monthly_df["Synthetic Index Dividend"],
}

div_log_levels = {
    "div_daily_log":   synthetic_div_daily_df["Log_Value"],
    "div_weekly_log":  synthetic_div_weekly_df["Log_Value"],
    "div_monthly_log": synthetic_div_monthly_df["Log_Value"],
}

div_log_returns = {
    "div_daily_log_ret":   synthetic_div_daily_df["Log_Returns"],
    "div_weekly_log_ret":  synthetic_div_weekly_df["Log_Returns"],
    "div_monthly_log_ret": synthetic_div_monthly_df["Log_Returns"],
}

div_levels_stats      = make_desc_table(div_levels, "Dividend level")
div_log_levels_stats  = make_desc_table(div_log_levels, "Dividend log-level")
div_log_returns_stats = make_desc_table(div_log_returns, "Dividend log-returns")

print("=== Dividend LEVELS ===")
print(div_levels_stats, "\n")
print("=== Dividend LOG-LEVELS ===")
print(div_log_levels_stats, "\n")
print("=== Dividend LOG-RETURNS ===")
print(div_log_returns_stats, "\n")


# ============================
# RISK-FREE
# ============================
# column-name mismatch (hyphen vs en-dash) caused KeyError.
# Use a small helper to locate the correct column name robustly.
def _find_rf_col(df):
    # common canonical candidates
    candidates = [
        "1-month Yield - US Treasury Securities",
        "1-month Yield – US Treasury Securities",
        "1-month Yield -- US Treasury Securities",
        "1-month Yield — US Treasury Securities",
        "1 month Yield - US Treasury Securities"
    ]
    for c in candidates:
        if c in df.columns:
            return c
    # fallback: look for a column containing both 'month' and 'treasury' (case-insensitive)
    for c in df.columns:
        cl = c.lower()
        if "month" in cl and "treasury" in cl:
            return c
    # final fallback: if there's only one numeric-month yield column, return the first match containing 'yield'
    for c in df.columns:
        if "yield" in c.lower() and "month" in c.lower():
            return c
    raise KeyError(f"Could not find a '1-month' risk-free column in dataframe columns: {list(df.columns)}")


rf_levels = {
    "rf_daily_level":   risk_free_daily_df[_find_rf_col(risk_free_daily_df)],
    "rf_weekly_level":  risk_free_weekly_df[_find_rf_col(risk_free_weekly_df)],
    "rf_monthly_level": risk_free_monthly_df[_find_rf_col(risk_free_monthly_df)],
}

rf_log_levels = {
    "rf_daily_log":   risk_free_daily_df["Log_Value"],
    "rf_weekly_log":  risk_free_weekly_df["Log_Value"],
    "rf_monthly_log": risk_free_monthly_df["Log_Value"],
}

rf_log_returns = {
    "rf_daily_log_ret":   risk_free_daily_df["Log_Returns"],
    "rf_weekly_log_ret":  risk_free_weekly_df["Log_Returns"],
    "rf_monthly_log_ret": risk_free_monthly_df["Log_Returns"],
}

rf_levels_stats      = make_desc_table(rf_levels, "Risk-free level")
rf_log_levels_stats  = make_desc_table(rf_log_levels, "Risk-free log-level")
rf_log_returns_stats = make_desc_table(rf_log_returns, "Risk-free log-returns")

print("=== Risk-free LEVELS ===")
print(rf_levels_stats, "\n")
print("=== Risk-free LOG-LEVELS ===")
print(rf_log_levels_stats, "\n")
print("=== Risk-free LOG-RETURNS ===")
print(rf_log_returns_stats, "\n")

=== Dividend LEVELS ===
                    count      mean       std       min       25%       50%  \
Dividend level                                                                
div_daily_level    8720.0  0.001871  0.010282  0.000000  0.000000  0.000000   
div_weekly_level   1246.0  0.013092  0.028165  0.000000  0.000410  0.002514   
div_monthly_level   287.0  0.056839  0.075660  0.000381  0.007747  0.023514   

                        75%       max  
Dividend level                         
div_daily_level    0.000134  0.448493  
div_weekly_level   0.012218  0.449044  
div_monthly_level  0.067910  0.453498   

=== Dividend LOG-LEVELS ===
                     count      mean       std        min       25%       50%  \
Dividend log-level                                                              
div_daily_log       2382.0 -6.519079  1.778922 -11.281414 -7.721611 -6.670531   
div_weekly_log      1066.0 -5.564421  1.886488 -11.281414 -6.665165 -5.528418   
div_monthly_log      287.0